# Econometría Aplicada II
## Tarea 1
Importar librerías

In [1]:
%%capture
# Clonar repo si estamos en colab
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/ArturoSbr/EmtrAp2-hw01
    # !pip install scipy==1.7.3
    %cd EmtrAp2-hw01/cod

# Libs
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.api import OLS
from matplotlib import pyplot as plt

Importar datos

In [2]:
d1 = pd.read_csv('../dat/baseline.csv')
d2 = pd.read_csv('../dat/endline.csv')
d3 = pd.read_csv('../dat/completa.csv')

### 1. Balance
Tabla de balance por grupo de acuerdo a `T_nap`

In [3]:
# Seleccionar 10 variables basales
X = ['time_in_office','age_','female_','education_','sleep_night','no_of_children_','act_inbed',
     'an_12_number_of_awakenings','an_13_average_awakening_length','unemployed']

# Inicializar lista
d = []

# Medias de variables basales
for x in X:
    # Grupos
    b, a = d1.groupby('T_nap')[x].apply(np.array)
    # t-test
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    # Agregar a lista
    d.append([x] + list(test))

# A tabla
t = pd.DataFrame(data=d, columns=['var','t','p']).sort_values('var')
t.round(3)

,var,t,p
6,act_inbed,1.255,0.210
1,age_,-0.249,0.804
7,an_12_number_of_awakenings,0.780,0.436
8,an_13_average_awakening_length,0.111,0.912
3,education_,-1.296,0.196
2,female_,-0.034,0.973
5,no_of_children_,0.533,0.594
4,sleep_night,0.612,0.541
0,time_in_office,-0.246,0.806
9,unemployed,0.491,0.624


Todos los p-values de las pruebas de diferencia de medias son mayores a 0.1. Esto significa que para toda variable independiente $X_j$, no podemos rechazar la hipótesis nula $\mu_{X_j}^C = \mu_{X_j}^T$. Las pruebas individuales sugieren que no existe ninguna diferencia entra las medias del grupo de control y el grupo de tratamiento.

Para evaluar la significancia de manera conjunta, uso el siguiente modelo de probabilidad lineal:
$$T_i = \beta_0 + X_i^T \beta + U_i$$
donde $X_i^T$ contiene todos los controles evaluados en la prueba anterior.

La prueba de hipótesis para determinar si al menos una de las variables indepentientes está relacionada con la asignación a tratamiento es:
$$H_0: \beta_1 = 0$$
$$H_1: \text{Al menos un coeficiente es distinto de cero}$$

In [4]:
# T_nap en función de controles
m = OLS(endog=d1['T_nap'], exog=d1[X].assign(const = 1)).fit()

# Tabla de resumen
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:                  T_nap   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.5121
Date:                Sat, 30 Apr 2022   Prob (F-statistic):              0.882
Time:                        15:45:51   Log-Likelihood:                -297.84
No. Observations:                 414   AIC:                             617.7
Df Residuals:                     403   BIC:                             662.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
time_in_offi

Al igual que en las diferencias de medias, todos los coeficientes estimados del modelo de probabilidad lineal tienen p-values menores a 0.1. Por ende, ninguno de las variables independientes tiene significancia individual para determinar la asignación a tratamiento.

La prueba que nos interesa se ve reflejada en el estadístico $F$. La prueba de significancia conjunta tiene un p-value de 0.882. Es decir, no hay evidencia de que alguno de los coeficientes sea distinto de cero.

Como conclusión, parece que el tratamiento sí fue asignado de manera aleatoria con base en los 10 controles que elegí.

### 2. Efectos de tratamiento
Efectos de tratamiento sobre la productividad de los trabajadores.

#### a) Estimadores de Neyman

In [12]:
# Función para estimador de Neyman
def neyman(frame, treatment_col, values_col):
    # Arreglos C y T
    m = frame[[treatment_col,values_col]].notna().all(axis=1)
    a, b = frame[m].groupby(treatment_col)[values_col].apply(np.array)
    # Diferencia de medias
    tau = np.mean(b) - np.mean(a)
    # Error estándar sobre-estimado
    bse = np.sqrt(np.var(a, ddof=1) / len(a) + np.var(b, ddof=1) / len(b))
    # t-stat
    t = tau / bse
    # p-value
    p = 2 * (1 - stats.norm().cdf(np.abs(t)))
    return [treatment_col, tau, bse, t, p]

# Diferencia de Neyman
prod_ney = neyman(d2, 'T_nap', 'productivity')
print(f'Prueba de Neyman sobre productividad:\n{prod_ney}')

Prueba de Neyman sobre productividad:
['T_nap', -170.53711387267913, 177.8423473054588, -0.9589229812614184, 0.3375975464907819]


#### b) Estimadores OLS sin controles

In [13]:
m = OLS(endog=d2['productivity'], exog=d2.assign(const = 1)[['const','T_nap']], missing='drop')
m = m.fit(cov_type='HC0')

# Diferencia OLS sin X
prod_ols = ['T_nap', m.params['T_nap'], m.bse['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']]
print(f'Estimación OLS sobre productividad:\n{prod_ols}')

Estimación OLS sobre productividad:
['T_nap', -170.53711387267884, 177.41291329606045, -0.9612440870529672, 0.3364294539166679]


#### c) Estimadores con controles
De acuerdo al paper, $X_i$ contiene `age_` en cuartiles, `female_` y la variable que indica si $i$ fue asignado a trabajar o a tomarse un break en vez de tomar una siesta.

Como esta pregunta usa la base con promedios durante los 20 días de estudio, la variable que indica la actividad asignada cada día a los individuos del grupo de control no está disponible. Por ende, usaré el siguiente model:
$$Y_i = \beta_0 + \sum_{q=1}^4I\big[\beta_q \times Q(age_i)\big] + \beta_f fem_i$$
donde $Q(age_i)$ asigna un cuartil a $i$ con base en su edad y $fem_i = 1$ si $i$ es mujer.

In [17]:
# Edad a cuartiles y luego a dummies
d2['age_q'] = pd.qcut(x=d2['age_'], q=4, labels=[f'q{i}' for i in range(1,5)])
d2 = pd.get_dummies(data=d2, prefix='age_', prefix_sep='', columns=['age_q'], )

# Tratamiento y controles
X = ['T_nap','const','age_q2','age_q3','age_q4','female_']

# Correr regresión
m = OLS(endog=d2['productivity'], exog=d2.assign(const = 1)[X], missing='drop')
m = m.fit(cov_type='HC0')

# Regresión con controles
prod_ctr = ['T_nap', m.params['T_nap'], m.bse['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']]
print(f'Estimación OLS+X sobre productividad:\n{prod_ctr}')

Estimación OLS+X sobre productividad:
['T_nap', -212.8637174329257, 168.48555935750844, -1.2633944312179985, 0.2064474644578932]


#### d) Resultados a tabla

In [26]:
# Concatenar resultados
t = pd.DataFrame(data=[prod_ney, prod_ols, prod_ctr],
                 columns=['Y','tau','se','t','p'],
                 index=['Neyman','OLS simple','OLS controles'])
t

,Y,tau,se,t,p
Neyman,T_nap,-170.537114,177.842347,-0.958923,0.337598
OLS simple,T_nap,-170.537114,177.412913,-0.961244,0.336429
OLS controles,T_nap,-212.863717,168.485559,-1.263394,0.206447


Los tres métodos indican que tomar una siesta tiene un efecto negativo sobre la productividad de los trabajadores. Sin embargo, todos los efectos estimados carecen de significancia. Es decir, el efecto promedio de tratamiento no es estadísticamente distinto de cero.

Mi modelo preferido es el OLS con controles porque redujo el p-value del coeficiente asociado al tratamiento.

#### e) Nuevas variables dependientes

In [28]:
# Crear índice de habilidades cognitivas
cog = ['corsi_measure','hf_measure','pvt_measure']
d2['cog'] = d2[cog].apply(lambda x: (x - x.mean()) / x.std()).mean(axis=1)

# Nuevas variables dependientes
Y = ['nap_time_mins','sleep_report','happy','cog','typing_time_hr']

# Correr regresiones
d = []
for y in Y:
    m = OLS(endog=d2[y], exog=d2.assign(const = 1)[X], missing='drop')
    m = m.fit(cov_type='HC0')
    d.append([m.params['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']])

# Resultados a tabla
t = pd.DataFrame(data=d, columns=['tau','t','p'], index=Y)

# Visualizar
t.transpose().round(3)

,nap_time_mins,sleep_report,happy,cog,typing_time_hr
tau,11.745,0.049,0.050,0.029,-0.018
t,39.444,1.119,1.422,0.531,-0.167
p,0.000,0.263,0.155,0.596,0.868


De acuerdo al modelo OLS con controles, el tratamiento:
1. Aumenta el promedio de minutos dormidos durante la siesta de 0 a 11.7 minutos
1. Aumenta el promedio de número de horas de sueño en 0.05 horas por día (pero no tiene significancia estadística)
1. Aumenta el promedio de la calificación de felicidad reportada en 0.05 puntos (pero no tiene significancia estadística)
1. Aumentar el índice promedio de desempeño cognitivo en 0.03 desviaciones estándar (pero no tiene significancia estadística)
1. Reducir el promedio de horas trabajadas en 0.02 unidades diarias (pero no tiene significancia estadística)

### 3. Fischer's Exact Test
#### a) Probar si el tratamiento tiene efecto nulo

In [ ]:
fet = stats.permutation_test(data=(d2.loc[d2['T_nap'].eq(1) & d2['productivity'].notna(), 'productivity'],
                                   d2.loc[d2['T_nap'].eq(0) & d2['productivity'].notna(), 'productivity']),
                             statistic=lambda x, y: np.mean(x) - np.mean(y),
                             n_resamples=1000,
                             random_state=42)

print('p-value:', round(fet.pvalue, 3))

p-value: 0.308


#### b) Conclusión
Los p-values del efecto promedio estimado de tratamiento bajo Neyman y de OLS sin controles son 0.338 y 0.336 respectivamente. Con la falsificación de Fischer, el p-value es 0.308. Estos p-values son muy parecidos entre sí, por lo que podemos concluir con un alto grado de certeza que el efecto de las siestas sobre la productividad no es estadísticamente significativo.

Cuando agregamos controles al modelo OLS, el p-value baja un poco, pero permance sin significancia estadística.

En resumen, todos los casos indican que las siestas no tienen un efecto estadísticamente significativo sobre la productividad de las personas.

### 4. Estratificación

In [29]:
# Crear casos con datos basales
d1[['e','s']] = d1[['earnings','sleep_report']].apply(lambda x: (x >= x.median()).astype(int), axis=0)

# Agregar casos a `d2`
d2 = d2.merge(d1[['pid','e','s']], on='pid')

#### a) Número de observaciones asignadas a tratamiento en cada estrato

In [31]:
# Número de observaciones asignadas a tratamiento
print('Observaciones asignadas a tratamiento por grupo:',
      d2.groupby(['e','s'])['T_nap'].sum(), sep='\n')

# Número de observaciones en cada grupo
print('Observaciones en cada grupo:',
      d2.groupby(['e','s']).size(), sep='\n')

Observaciones asignadas a tratamiento por grupo:
e  s
0  0    48
   1    54
1  0    54
   1    49
Name: T_nap, dtype: int64
Observaciones en cada grupo:
e  s
0  0     97
   1    110
1  0    110
   1     97
dtype: int64


El 50% de la población fue asignada a tratamiento. Sin embargo, el número de personas en cada grupo no es constante (51.25), sino que a veces está por encima de este número y a veces está por debajo.

#### b) Efectos por estrato y agregados
Efectos por estrato

In [34]:
# Inicializar lista
d = []

# Efecto por esstrato a cada variable
for y in Y:
    for e, s in [(0,0),(0,1),(1,0),(1,1)]:
        # Máscara
        m = d2['e'].eq(e) & d2['s'].eq(s)
        # ATE Neyman
        d.append([e, s, m.sum(), m.sum() / len(d2), y] + neyman(d2[m], 'T_nap', y))

# Resultados a tabla
t = pd.DataFrame(data=d,
                 columns=['aboveEarn','aboveSleep','ng','wg','depvar','indvar','tau','se','t','p'])
t = t.set_index(['depvar','aboveEarn','aboveSleep'])

# Visualizar resultados
t[['tau']].round(3)

tau
depvar         aboveEarn aboveSleep        
nap_time_mins  0         0           11.516
                         1           12.430
               1         0           10.873
                         1           12.048
sleep_report   0         0            0.005
                         1            0.167
               1         0            0.020
                         1           -0.016
happy          0         0           -0.042
                         1            0.043
               1         0            0.004
                         1            0.203
cog            0         0            0.116
                         1            0.169
               1         0           -0.027
                         1           -0.148
typing_time_hr 0         0            0.295
                         1           -0.256
               1         0           -0.274
                         1            0.219

Efectos agregados

In [37]:
t['tau'].multiply(t['wg']).groupby(t.index.get_level_values(0)).sum()

depvar
cog                0.030237
happy              0.050259
nap_time_mins     11.712448
sleep_report       0.047170
typing_time_hr    -0.020488
dtype: float64

#### c) Efectos estratificados con OLS
$$Y_i = \beta_0 + \beta_1 T_i + \beta_2 e_i + \beta_3 e_i T_i + \beta_4 s_i + \beta_5 s_i T_i + \beta_6 e_i s_i + \beta_7 e_i s_i T_i + U_i$$

In [58]:
# Inicializar lista
d = []

# Regresión para todas las depvar
for y in Y:
    # Modelo
    formula = f'{y} ~ T_nap + e + I(e*T_nap) + s + I(s*T_nap) + I(e*s) + I(e*s*T_nap)'
    m = OLS.from_formula(formula=formula, data=d2).fit(cov_type='HC0')
    # Results table
    res = pd.concat([m.params, m.bse, m.pvalues, m.tvalues], axis=1).assign(depvar = y)
    d.append(res)

# Tabla conjunta
d.append()

In [66]:
t = pd.concat(d, axis=0).reset_index()
t.columns = ['beta','value','bse','t','p','depvar']
t
# t[t['beta'].isin(['T_nap','I(e * T_nap),I(s * T_nap)'])]

,beta,value,bse,t,p,depvar
0,Intercept,4.936881e-15,1.368786e-15,3.100462e-04,3.606758,nap_time_mins
1,T_nap,1.151558e+01,5.777160e-01,2.108338e-88,19.932934,nap_time_mins
2,e,-4.770774e-15,1.251998e-15,1.386708e-04,-3.810527,nap_time_mins
3,I(e * T_nap),-6.430445e-01,8.184395e-01,4.320457e-01,-0.785696,nap_time_mins
4,s,-4.929853e-16,1.666892e-15,7.674201e-01,-0.295751,nap_time_mins
5,I(s * T_nap),9.144070e-01,8.164858e-01,2.627436e-01,1.119930,nap_time_mins
6,I(e * s),4.493216e-16,1.458008e-15,7.579492e-01,0.308175,nap_time_mins
7,I(e * s * T_nap),2.611671e-01,1.196587e+00,8.272264e-01,0.218260,nap_time_mins
8,Intercept,6.457659e-01,1.108339e-02,0.000000e+00,58.264309,sleep_report
9,T_nap,4.795823e-03,1.555259e-02,7.578071e-01,0.308362,sleep_report


### 5. Atrición
#### a) Reportar atrición

In [ ]:
t = d2.groupby('T_nap')['drop_indicator'].agg(['size','sum'])
t['pct'] = t['sum'].div(t['size']) * 100
t.round(2)

#### b) Nuevo balance
Validez interna

In [ ]:
# Variables dependientes
X = ['time_in_office','age_','female_','education_','sleep_night','no_of_children_','act_inbed',
     'an_12_number_of_awakenings','an_13_average_awakening_length','unemployed']

# Tabla de balance
d = []
for x in X:
    b, a = d2[d2['drop_indicator'].eq(0)].groupby('T_nap')[x].apply(np.array)
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    d.append([x] + list(test))

t = pd.DataFrame(data=d, columns=['variable','t-stat','p-value'])
t.sort_values('variable').round(3)

In [ ]:
m = OLS(endog=d2.loc[d2['drop_indicator'].eq(0), 'T_nap'],
        exog=d2.loc[d2['drop_indicator'].eq(0), X].assign(const = 1),
        missing='drop').fit(cov_type='HC0')
print(f'p-value de significancia conjunta: {round(m.f_pvalue, 3)}')

Antes de la atrición, ninguna de las 10 variables tenía diferencias entre tratamiento y control que fueran estadísticamente significativas. Después de la atrición, los p-values son más chicos y algunas diferencias tienen significancia a nivel individual. Por ejemplo, las diferencias en `education_` y `no_of_children_` ahora son significativas al 1 y 5 porciento de confianza.

Al hacer la prueba de significancia conjunta usando el modelo $T_i = X_i^T \beta + U_i$, el p-value es aproximadamente 0.1. Es relativamente improbable que todos los coeficientes sean simultáneamente igual a cero, pero la significancia de la prueba cayó tras la atrición.

Representatividad externa

In [ ]:
# Probar si diferencia es significativa Baseline VS Endline
d = []
for x in X:
    a, b = d2[x], d2.loc[d2['drop_indicator'].eq(0), x]
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    d.append([x] + list(test))

# Resultados a tabla
t = pd.DataFrame(data=d, columns=['variable','t','p'])
t.sort_values('variable').round(3)

La atrición parece no haber afectado la representatividad externa de la muestra. Ninguna de las diferencias son significativas individualmente.

#### c) Conclusión
La atrición fue sistemática entre el grupo de tratamiento y de control. Es decir, parece que el nivel de educación y el número de hijos determinan si alguien abandona o no el experimento. Esto nos lleva a un problema de validez interna porque los grupos de tratamiento y control después de la atrición no están balanceados.

Sin embargo, parece que la atrición no afectó la validez externa de la muestra, pues parece que las personas que abandonaron el estudio no afectaron las distribuciones de las variables de control. Ninguna de las 10 variables muestra una diferencia significativa antes y después de la atrición.

### 5. Lee Bounds
#### a) Perfiles
- Always Respondents: $S_i$ = 1 sin importar $T_i$
- Never Respondents: $S_i$ = 0 sin importar $T_i$
- Selective Respondents: $T_i = 0 \implies S_i = 0$, $T_i = 1 \implies S_i = 1$
- Counter-Selective Respondents: $T_i = 0 \implies S_i = 1$, $T_i = 1 \implies S_i = 0$

El supuesto de monotonicidad es que no existe alguno de los dos grupos de respuesta selectiva. En el contexto de este experimento, tiene sentido asumir que los Counter-Selective Respondents no existen porque el tratamiento es algo *bueno*. Es decir, si a alguien le toca tomar una siesta durante sus horas de trabajo, es razonable pensar que el tratamiento es algo deseable y por ende no incentivaría a los individuos a abandonar el experimento.

In [ ]:
# Columna S_i
d2['S'] = 1 - d2['drop_indicator']

# Casos
t = d2.groupby(['T_nap','S']).size()
t.unstack().transpose()

In [ ]:
# P(AR|T=0)
par = 128 / (81 + 128)

# P(SR|T=1)
psr = 182 / (182 + 23) - par

# Probabilidades
print(f'P(AR) = {par}', f'P(SR) = {psr}', f'P(NR) = {1 - (par + psr)}', sep='\n')

#### b) Lee Bounds

In [ ]:
# Arreglos de S_i = 1
a, b = d2[d2['S'].eq(1)].groupby('T_nap')['productivity'].apply(np.array)

# Lower bound
lb = b[b <= np.quantile(b, 1 - psr).mean()].mean() - a.mean()

# Upper bound
ub = b[b >= np.quantile(b, psr)].mean() - a.mean()

# Bounds
print(f'El ATE de los AR está en [{lb, ub}]')

#### c) Comparación
Los resultados de la pregunta 2 no tienen por qué estar centrados en los Lee Bounds porque estiman el efecto para toda la población, mientras que el intervalo de esta pregunta acota el efecto de tratamiento para los Always Respondents.

Lo único que sí podemos ver es que el ATE observado en la muestra completa es mayor al ATE de los Always Respondents.

## 2. Matching